# Prepare data and import libraries

In [ ]:
#Mount Google drive to colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#import libraries
import scipy.ndimage as nd
import scipy.io as io
import matplotlib.pyplot as plt
import skimage.measure as sk

import numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import torch
import os

import torch.optim as optim

import torch.nn as nn

In [ ]:
#go to root folder of project
cd drive/My Drive/3D-GAN

In [ ]:
#unzip training data
if not os.path.exists('train'):
  !unzip train.zip

In [ ]:
root_dir = os.getcwd()
data_dir = os.path.join(root_dir, 'train')

In [ ]:
def getVoxelFromMat(path, cube_len=64):
    voxels = io.loadmat(path)['instance']
    voxels = np.pad(voxels, (1, 1), 'constant', constant_values=(0, 0))
    if cube_len != 32 and cube_len == 64:
        voxels = nd.zoom(voxels, (2, 2, 2), mode='constant', order=0)
    return voxels

def plotFromVoxels(voxels):
    z, x, y = voxels.nonzero()
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x, y, -z, zdir='z', c='red')
    plt.show()

class ShapeNetDataset(Dataset):


    def __init__(self, root, cube_len = 32):
        """Set the path for Data.
        Args:
            root: image directory.
        """
        self.root = root
        self.listdir = os.listdir(self.root)
        self.cube_len = cube_len


    def __getitem__(self, index):
        with open(os.path.join(self.root, self.listdir[index]), "rb") as f:
            volume = np.asarray(getVoxelFromMat(f, self.cube_len), dtype=np.float32)
        return torch.FloatTensor(volume)

    def __len__(self):
        return len(self.listdir)

dataset = ShapeNetDataset(data_dir)

sample = dataset.__getitem__(0)
plotFromVoxels(sample.numpy())

print('voxel shape = {}'.format(sample.shape))


#Define Generator and Discriminator models

In [ ]:
class Generator(nn.Module):
  def __init__(self, channel_sizes = [512, 256, 128, 64], z_shape = 200):
    super(Generator, self).__init__()
    #first layer

    #second layer
   
    
    #third layer
   
    
    #fourth layer
   
    
    #final layer: conv transpose + sigmoid
   
    
  def forward(self, x):
    #apply layer 1

    #apply layer 2

    #apply layer 3

    #apply layer 4

    #apply layer 5




z = torch.randn((4, 200, 1, 1, 1))
net = Generator()
output = net(z)

print(output.shape)

In [ ]:
class Discriminator(nn.Module):
  def __init__(self, channel_sizes = [32, 64, 128, 256]):
     super(Discriminator, self).__init__()
     #layer 1
   
     #layer 2
     
  
     #layer 4

     #layer 5 conv + sigmoid


  def forward(self, x):
    #apply layer 1

    #apply layer 2

    #apply layer 3

    #apply layer 4

    #apply layer 5


dnet = Discriminator()
output = dnet(output)
print(output.shape)

In [ ]:
batch_size = 32
g_lr = 0.0025
d_lr = 0.001
nepochs = 1000

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
dataset = ShapeNetDataset(root = data_dir)
dataloader = DataLoader(dataset, batch_size= batch_size, shuffle = True, drop_last = True, num_workers = 2)

gen = Generator().to(device)
disc = Discriminator().to(device)

g_optimizer = optim.Adam(gen.parameters(), lr = g_lr)
d_optimizer = optim.Adam(disc.parameters(), d_lr)

criterion = nn.BCELoss()

In [ ]:
from tqdm import tqdm
for e in range(nepochs):
  for i, data in enumerate(tqdm(dataloader, leave=False, desc='Batches'), 0):
  #TODO
  # move data to device

  #TODO
  # generate random codes
  random_codes = ...
  #TODO
  # generate samples from random codes

  fake_data = ...

  #labels
  real_labels = torch.zeros(batch_size).to(device)
  fake_labels = torch.zeros(batch_size).to(device)

  #TODO
  # compute discriminator outputs
  real_preds = ...
  fake_preds = ...
  #train discriminator
  d_real_acu = torch.le(real_preds.squeeze(), 0.5).float()
  d_fake_acu = torch.ge(fake_preds.squeeze(), 0.5).float()
  d_total_acu = torch.mean(torch.cat((d_real_acu, d_fake_acu),0))

  if d_total_acu < 0.8:
    #TODO 
    #compute discriminator loss, back propagate and optimize discriminator

  #generate fake shapes
  fake_data = gen(random_codes)
  fake_preds = disc(fake_data).view(-1,)

  #compute generator loss, back propagate and optimize generator
